<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/FlexinATR_TELEGRAM_updates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading required packages
!pip install yfinance
!pip install numpy
!pip install pandas
!pip install hurst
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import of the necessary modules
from IPython.display import clear_output 
from time import sleep
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [3]:
def telegram(message):

    apiToken = '6183952601:AAFbfrKfZb3iKX6GfYZQnatxo0lg1S4lL8Y'
    chatID = '@flexinflexinbot1'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

In [4]:
class Flexin:
  
  def __init__(self, wallet):
    '''
    Symbol of any pair, stock or commodity
    :param symbol: symbol
    '''
    self.wallet = wallet # Self wallet variable

  # Objects
  def run(self):
    # Packages needed for this function
    import numpy as np
    import yfinance as yf
    from hurst import compute_Hc
    '''
    Returns data for any symbol - The first collection is made for the daily
    chart and then the hourly chart.
    :param symbol: asset symbol for analysis 
    '''
    for symbol in wallet:
      # BBs variables
      n, k = 20, 2 # Moving average periods, Factor to shift the bands
      # ATR variables
      p = 14 # Period for calculating the ATR (days)
      # Verify signals variables
      verifyBuy, verifySell = False, False # Verify signals
      print(f'analyzing {symbol}...') # Show the symbol
      #------------------------------------------------------------------------- 
      # Daily filter - For period => 'max' and interval='1d'
      #-------------------------------------------------------------------------
      d = yf.Ticker(symbol).history(period='max', interval='1d') # Daily data
      d.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
      # Hurst exponent filter (https://github.com/Mottl/hurst) -----------------
      H = lambda x: compute_Hc(x)[0] # Compute hurst on anonymous lambda func.
      d['Hurst'] = d['Close'].rolling(window=1024).apply(H) # Hurst D average
      hurstDaily = d['Hurst'][-1] # Current daily hurst
      # Bollinger Bands --------------------------------------------------------
      d['Standard Deviation'] = d['Close'].rolling(n).std() # Stdv
      d['Middle Band'] = d['Close'].rolling(n).mean() # Middle B.
      d['Upper Band'] = d['Middle Band'] + d['Standard Deviation']*k # Upper B.
      d['Lower Band'] = d['Middle Band'] - d['Standard Deviation']*k # lower B.
      # ATR daily --------------------------------------------------------------
      d['H-L'] = d['High']-d['Low'] # diff between the maximum & minimum
      d['|H-Cp|'] = np.abs(d['High']-d['Close'].shift(1)) # diff of maximum
      d['|L-Cp|'] = np.abs(d['Low']-d['Close'].shift(1)) # diff of minimum
      # TR - maximum value among these three possibilities (in each line)
      d['TR'] = d[['|H-Cp|', 'H-L', '|L-Cp|']].max(axis=1) # Maximum value diff
      smaTR = d['TR'].rolling(window=p, min_periods=p).mean()[:p] # TR sma
      restTR = d['TR'][p:] # Rest of indicator
      # ATR - Average True Range
      d['ATR'] = pd.concat([smaTR, restTR]).ewm(alpha=1/p, adjust=False).mean()
      #-------------------------------------------------------------------------
      # Hour filter - For period => '1y' and interval='1h'
      #-------------------------------------------------------------------------
      h = yf.Ticker(symbol).history(period='1y', interval='1h') # Hour data
      h.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
      # Hurst exponent filter (https://github.com/Mottl/hurst)
      H = lambda x: compute_Hc(x)[0] # Compute hurst
      h['Hurst'] = h['Close'].rolling(window=1024).apply(H) # Hurst H average
      hurstHour = h['Hurst'][-1] # Currente hour hurst
      # Bollinger Bands
      h['Standard Deviation'] = h['Close'].rolling(n).std() # Stdv
      h['Middle Band'] = h['Close'].rolling(n).mean() # Middle
      h['Upper Band'] = h['Middle Band'] + h['Standard Deviation']*k # Upper
      h['Lower Band'] = h['Middle Band'] - h['Standard Deviation']*k # Lower
      #-------------------------------------------------------------------------
      # Strategy
      #-------------------------------------------------------------------------
      if 1>d['Hurst'][-1]>0.5 and 1>h['Hurst'][-1]>0.5:
        print(f'H© = {hurstDaily.round(4)} 🏂! w/ persistent behavior.')
        # BUY/LONG -------------------------------------------------------------
        if d['Open'][-1]>d['Middle Band'][-1]<d['Close'][-1]:
          if d['Close'][-1]>d['Open'][-1]:
            if h['Open'][-1]>h['Middle Band'][-1]:
              if h['Close'][-1]>h['Upper Band'][-1]:
                verifyBuy = True
                if verifyBuy == True:
                  price = d['Close'][-1]
                  ATR = d['ATR'][-1]
                  buyInfo = {
                      'Symbol': symbol,
                      'Type': 'Buy',
                      'Take Profit': price+ATR,
                      'Stop Loss': price-ATR}
                  print(f'📈 BUY: {verifyBuy} for {symbol} at price {price}.\n{buyInfo}')
                  telegram(f'📈 BUY: {verifyBuy} for {symbol} at {price}.\n{buyInfo}')
        else:
          # SELL/SHORT ---------------------------------------------------------
          if d['Open'][-1]<d['Middle Band'][-1]>d['Close'][-1]:
            if d['Close'][-1]<d['Open'][-1]:
              if h['Close'][-1]<h['Lower Band'][-1]:
                if h['Open'][-1]<h['Middle Band'][-1]:
                  verifySell = True
                  if verifySell == True:
                    price = d['Close'][-1]
                    ATR = d['ATR'][-1]
                    sellInfo = {
                      'Symbol': symbol,
                      'Type': 'Sell',
                      'Take Profit': price-ATR,
                      'Stop Loss': price+ATR}
                    print(f'📉 BUY: {verifySell} for {symbol} at price {price}.\n{sellInfo}')
                    telegram(f'📉 BUY: {verifySell} for {symbol} at {price}.\n{sellInfo}')
      else:
        print(f'H© = {hurstDaily.round(4)} 💤! w/ anti-persistent.')
      print('')

In [5]:
wallet = ['EURUSD=X',
          'JPY=X',
          'GBPUSD=X',
          'CHF=X',
          'AUDUSD=X',
          'CAD=X',
          'NZDUSD=X',
          '^HSI']

In [ ]:
while True:
  Flexin(wallet).run()
  clear_output()

analyzing EURUSD=X...
H© = 0.5529 💤! w/ anti-persistent.

analyzing JPY=X...
H© = 0.5739 🏂! w/ persistent behavior.

analyzing GBPUSD=X...
H© = 0.5335 💤! w/ anti-persistent.

analyzing CHF=X...
H© = 0.4627 💤! w/ anti-persistent.

analyzing AUDUSD=X...
